<a href="https://colab.research.google.com/github/soujanya1208/NNDL/blob/main/assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 1.Convert the above paragraph into vectors using:<br>
'''i) Word2vec
    ii) USE
iii)ELMO
iv) GP2
v) Sentence-BERT'''

'i) Word2vec\n    ii) USE\niii)ELMO\niv) GP2\nv) Sentence-BERT'

In [8]:
my_string='''paragraph is a series of sentences that are organized and coherent, and are all  related  to  a  single  topic.  Almost  every  piece  of  writing  you  do  that  is 
longer  than  a  few  sentences  should  be  organized  into  paragraphs.  This  is because paragraphs show a reader where the subdivisions of an essay begin 
and end, and thus help the reader see the organization of the essay and grasp its main points.

Paragraphs  can  contain  many  different  kinds  of  information.  A  paragraph could  contain  a  series  of  brief  examples  or  a  single  long  illustration  of  a 
general  point.  It  might  describe  a  place like kolkata,  character,  or  process;  narrate  a series of events; compare or contrast two or more things; classify items into 
categories;  or  describe  causes  and  effects.  Regardless  of  the  kind  of information they contain, all paragraphs share certain characteristics. One 
of the most important of these is a topic sentence.'''

In [ ]:
import numpy as np
import nltk
import re
import gensim
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim import corpora,models,similarities
from nltk.tokenize import PunktSentenceTokenizer
nltk.download('punkt')

In [ ]:
#code to convert paragraph to sentences
def essay_to_sentences(paragraph):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(paragraph.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append((raw_sentence))
    return sentences

sentences=essay_to_sentences(my_string)

sentences

In [ ]:
wordvecs=[nltk.word_tokenize(sent) for sent in sentences]
from nltk.corpus import stopwords
nltk.download('stopwords')
stops=list(set(stopwords.words("english")))
for i in wordvecs:
  for j in i:
    if j in stops:
      i.remove(j)
    elif len(j)==1:
      i.remove(j)

model=gensim.models.Word2Vec(wordvecs,min_count=1,size=32)

In [ ]:
#printing vector form of word 'paragraph'
model['paragraph']

In [ ]:
#finding similar words of word 'sentence' in given paragraph
model.most_similar('sentence')

In [ ]:
import tensorflow_hub as hub
use= hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
#converting to vectors
embeddings=use(sentences)
print(embeddings)

In [ ]:
print("shape= ",embeddings[0].shape)
#each sentence is converted into vector having 512 values
print("The sentence: ",sentences[0],"\n is converted as : \n{}".format(embeddings[0]))

In [ ]:
bert = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")
embeddings2=bert(sentences)
print(embeddings2)

In [ ]:
print("shape=",embeddings2[0].shape)
#each sentence is converted into vector having 128 values
print("The sentence: ",sentences[0],"\n is converted as : \n{}".format(embeddings2[0]))

In [ ]:
!pip install "tensorflow>=2.0.0"
!pip install --upgrade tensorflow-hub

In [ ]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
#1024 sized vectors
elmo=hub.Module("https://tfhub.dev/google/elmo/3",trainable=True)
embeddings=elmo(
    sentences,
    signature="default",
    as_dict=True)["elmo"]
init=tf.initialize_all_variables()
sess=tf.Session()
sess.run(init)
print("\n\n")
print(sess.run(embeddings[0]))
print("shape=",embeddings[0].shape)

In [ ]:
import torch
import transformers

In [ ]:
gptokenizer=transformers.GPT2Tokenizer.from_pretrained('gpt2-large')
model=transformers.GPT2LMHeadModel.from_pretrained('gpt2-large')
output=gptokenizer.encode(my_string,add_special_tokens=False,return_tensors="pt")

In [ ]:
print("shape=",output.shape)
output

2. Find named entities (NER) for the above paragraph?

In [ ]:
import spacy
from spacy import displacy
ner=spacy.load('en')
result=ner(my_string)

for word in result.ents:
  print(word.text,word.label_)

In [ ]:
spacy.explain('GPE')

In [ ]:
#better visualisation of entity recognition
displacy.render(result,style="ent",jupyter=True)

In [ ]:
# finding named entities of another paragraph
resultss=ner("Cricket was introduced to India by British sailors in the 18th century, and the first cricket club was established in 1792. India's national cricket team did not play its first Test match until 25 June 1932 at Lord's, becoming the sixth team to be granted test cricket status. From 1932 India had to wait until 1952, almost 20 years for its first Test victory. In its first fifty years of international cricket, India didn't gain much success, winning only 35 of the first 196 Test matches it played. The team, however, gained strength in the 1970s with the emergence of players like Gavaskar, Viswanath, Kapil Dev, and the Indian spin quartet.")
for word in resultss.ents:
  print(word.text,word.label_)

In [ ]:
#better visualisation of entity recognition
displacy.render(resultss,style="ent",jupyter=True)

In [ ]:
#3. Find similar sentences(repeated sentences) from the above paragraph?
pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
se_embeddings = sbert_model.encode(sentences)
q1_vec= sbert_model.encode(sentences[0])

#cosine similarity function
#identifies similarity between 2 sentences
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

for sent in sentences:
  sim = cosine(q1_vec, sbert_model.encode([sent])[0])
  #if similarity ==1 => repeated sentence
  #if similarity > 0.6 => similar sentence
  if sim>0.6:
    print("Sentence1 =",sentences[0],"\n \nSentence2=", sent, "\n\nsimilarity = ", sim,end="\n ----------------------------- \n")

In [ ]:
#4. Explain POS tagging with HMM?